In [15]:
from tg.grammar_ru.ml.corpus import CorpusReader
from pathlib import Path
import os

base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'corpus', 'example', 'corpus', 'example.base.zip'))
reader = CorpusReader(Path(base_path))
db = reader.get_bundles().first()

In [16]:
# --- getting pronouns block ---
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

def _get_pronoun_filter():
    return lambda x: ((x.normal_form == 'он') |
                      (x.normal_form == 'она') |
                      (x.normal_form == 'оно'))

def _get_candidate_filter():
    return lambda x: ((x.number == 'sing') &
                      ((x.POS == 'NOUN') |
                       (x.POS == 'PRON') |
                       (x.POS == 'ADJF') |
                       (x.POS == 'ADJS') |
                       (x.POS == 'NPRO') |
                       (x.POS == 'PRCL') |
                       (x.POS == 'PRTF') |
                       (x.POS == 'PRTS') |
                       (x.POS == 'ADVB')))

pmf = PyMorphyFeaturizer()
pmf.featurize(db)
morphology_df = db.data_frames['pymorphy']
pronouns_df = morphology_df.loc[_get_pronoun_filter(), ['gender', 'case']]
pronouns_df['word_id'] = pronouns_df.index
pronouns_df = pronouns_df.add_prefix('pronoun_')
pronouns_df

,pronoun_gender,pronoun_case,pronoun_word_id
word_id,,,
10,femn,accs,10
18,femn,nomn,18
30,femn,datv,30
48,masc,gent,48
53,femn,accs,53
56,femn,nomn,56
83,femn,nomn,83
113,femn,nomn,113
122,femn,nomn,122


In [18]:
# --- merge with antecedent candidates block
antecedent_candidates_df = morphology_df.loc[_get_candidate_filter(), ['POS', 'animacy', 'gender', 'case']]
antecedent_candidates_df['word_id'] = antecedent_candidates_df.index
antecedent_candidates_df = antecedent_candidates_df.add_prefix('candidate_')
merged_df = pronouns_df.merge(antecedent_candidates_df, how='cross')
merged_df = merged_df[(merged_df['pronoun_word_id'] > merged_df['candidate_word_id']) & 
                      (merged_df['pronoun_gender'] == merged_df['candidate_gender'])]
merged_df = merged_df.drop(columns=['pronoun_gender', 'candidate_gender']).reset_index(drop=True)
merged_df['candidate_distance'] = merged_df.groupby(['pronoun_word_id']).cumcount(ascending=False)
merged_df = merged_df[merged_df['candidate_distance'] < 10]
merged_df

,pronoun_case,pronoun_word_id,candidate_POS,candidate_animacy,candidate_case,candidate_word_id,candidate_distance
0,accs,10,NOUN,anim,nomn,0,0
1,nomn,18,NOUN,anim,nomn,0,1
2,nomn,18,NPRO,NaN,accs,10,0
3,datv,30,NOUN,anim,nomn,0,2
4,datv,30,NPRO,NaN,accs,10,1
...,...,...,...,...,...,...,...
3085,nomn,921,NOUN,anim,nomn,907,4
3086,nomn,921,NOUN,inan,nomn,914,3
3087,nomn,921,NPRO,NaN,datv,917,2
3088,nomn,921,NOUN,inan,loct,918,1


In [19]:
#--- what if we choose the last candidate?
last_candidates_df = merged_df[merged_df['candidate_distance'] == 0]
pair_indices = last_candidates_df[['pronoun_word_id', 'candidate_word_id']].reset_index(drop=True)
len(pair_indices)
pair_indices.head()

,pronoun_word_id,candidate_word_id
0,10,0
1,18,10
2,30,18
3,48,38
4,53,30


In [20]:
# не на ту напали — взять в работу указательные местоимения
pronouns = db.src[db.src['word_id'].isin(pair_indices['pronoun_word_id'])][['word_id', 'word']]
last_antecedents = db.src[db.src['word_id'].isin(pair_indices['candidate_word_id'])][['word_id', 'word']]
pair_indices = pair_indices.merge(pronouns, left_on='pronoun_word_id', right_on='word_id')
pair_indices = pair_indices.merge(last_antecedents, left_on='candidate_word_id', right_on='word_id')
pair_indices = pair_indices.drop(columns={'word_id_x', 'word_id_y'})
pair_indices

,pronoun_word_id,candidate_word_id,word_x,word_y
0,10,0,ее,Лилия
1,18,10,она,ее
2,30,18,Ей,она
3,48,38,него,злодея
4,53,30,ее,Ей
5,56,53,она,ее
6,83,82,она,спальни
7,113,110,она,вороны
8,122,118,она,Голова
9,145,142,Ее,Лилия


In [22]:
mistakes = pair_indices.loc[pair_indices.index.isin([6, 7, 8, 16, 17, 20, 23, 25, 26, 27, 34, 40, 39, 40, 41, 42, 43, 45, 46])]
mistakes

,pronoun_word_id,candidate_word_id,word_x,word_y
6,83,82,она,спальни
7,113,110,она,вороны
8,122,118,она,Голова
16,299,293,она,ту
17,335,312,Ее,поразившая
20,374,372,ее,Каролина
23,435,433,Она,победительницей
25,493,489,Она,ее
26,515,504,ней,Лилия
27,560,556,нее,Лилия


#### Ошибочный референс на причастие:
21. ...С простым грабителем **она** бы в два счета разделалась! Нет. Это наверняка порча, **~~поразившая~~** споенец — и именно оттуда исходит страх. Это был не разбой — не водились грабители в тихих, сонных Чисовицах. **Ее** хотел убить...
28. Через некоторое время **Лилии** удалось нащупать едва заметную ниточку, **~~соединяющую~~** **ее** с колдуном.

Вероятно, ситуации, когда причастие выступает в роли подлежащего или дополнения, очень редкие и сейчас причастия лучше выбрасывать, но последний кандидат всё равно будет неправильным.

#### При подстановке предмет будет совершать нетипичные действия, что можно было бы отсекать словарём сочетаний:
7. Закричав, **Лилия** резко села, открыла глаза и осмотрелась по сторонам. Странно. Вместо привычного потолка своей **~~спальни~~** **она** увидела небо
9. ...которых **она** напугала своим вскриком. **~~Голова~~** кружилась, и **она** чувствовала тревогу...
сложный случай:
40. **Лилии** оставалось полагаться на интуицию, как и подобает воительнице. Нет нужды видеть цель, если чувствуешь, где **~~она~~**. **Она** прошла в самый конец главного зала...

#### Неправильно определено число в данном контексте:
8. ...спина **девушки** опиралась не на изголовье кровати, а на глухую кирпичную стену. Недалеко каркали **~~вороны~~**, которых **она** напугала...
17. ...**Лилия** поняла, что цела — ни **~~ран~~**, ни ушибов. Прикоснувшись ко лбу, **она** мысленно обратилась...
35. ...**она** остановит его сама. Неужели колдун думал, что **~~двери~~** станут для **нее** преградой?

#### Неправильно определён род в контексте:
44. Сила, страсть **ее** были достаточны, чтобы **~~любой~~** рухнул навзничь. **Она** ожидала услышать...

#### Вероятно, последний кандидат слишком близко к местоимению:
27. Битва началась неожиданно, но **Лилия** твердо намеревалась выйти из нее **~~победительницей~~**.**Она** прислушалась к тревоге...

#### Местоимению не в именительном падеже поставлено в соответствие подлежащее из того же предложения:
24. ...**Она** была звездой дуэльного клуба Академии. **~~Каролина~~**, **ее** учительница...
18. ...**она** мысленно обратилась к своему споенцу. **~~Волна~~** страха пробежала по **ее** телу...
43. ...**Лилия** выкрикнула атакующее заклинание. **~~Сила~~**, страсть **ее** были достаточны...
45. **Она** ожидала услышать вопль боли, сдавленную брань, проклятья, просто удар тела об пол, в худшем случае — контратакующее заклинание, но не произошло вообще ничего. Тишина. **~~Темнота~~** в **ее** глазах вдруг замерцала...
46. **Лилия** остановилась — что-то пошло совсем не так. **~~Пульсация~~** усиливалась, учащалась, мелькала быстрее, чем билось **ее** сердце.
47. ...быстрее, чем билось **ее** сердце. В ушах стоял звон, становящийся все выше. **~~Тревога~~**, таившаяся в **ней**...
49. Лилия так и не смирилась с тем, что **она — жертва**, но успела осознать, что **~~судьба~~** не уготовила **ей** роли героини.
возможны проблемы с обработкой:
26. **Битва** началась неожиданно, но **~~Лилия~~** твердо намеревалась выйти из **нее** победительницей.

#### ?
50. ...судьба не уготовила **ей** роли **~~героини~~**. **Она** кубарем покатилась вниз...

In [7]:
#--- add pronoun parent index block
from tg.grammar_ru.ml.features import SlovnetFeaturizer

slvnt = SlovnetFeaturizer()
slvnt.featurize(db)
slovnet = db['slovnet']

parent_ids = slovnet[slovnet.index.isin(merged_df['pronoun_word_id'])]['syntax_parent_id']
parent_df = (parent_ids.to_frame()
             .reset_index()
             .rename(columns={'syntax_parent_id': 'pronoun_parent_id', 'word_id': 'pronoun_word_id'}))
merged_df = merged_df.merge(parent_df, on='pronoun_word_id')
merged_df

,pronoun_case,pronoun_word_id,candidate_POS,candidate_animacy,candidate_case,candidate_word_id,candidate_distance,pronoun_parent_id
0,accs,10,NOUN,anim,nomn,0,0,9
1,nomn,18,NOUN,anim,nomn,0,1,19
2,nomn,18,NPRO,NaN,accs,10,0,19
3,datv,30,NOUN,anim,nomn,0,2,31
4,datv,30,NPRO,NaN,accs,10,1,31
...,...,...,...,...,...,...,...,...
428,nomn,921,NOUN,anim,nomn,907,4,922
429,nomn,921,NOUN,inan,nomn,914,3,922
430,nomn,921,NPRO,NaN,datv,917,2,922
431,nomn,921,NOUN,inan,loct,918,1,922


In [23]:
# Тревога, таившаяся в ней, вскипела в панику.
slovnet.iloc[870:880]

,POS,Animacy,Case,Gender,Number,Aspect,Mood,Tense,VerbForm,Voice,Degree,Person,Polarity,Variant,Foreign,relation,syntax_parent_id
word_id,,,,,,,,,,,,,,,,,
870,NOUN,Inan,Nom,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nsubj,876
871,PUNCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,punct,872
872,VERB,NaN,Nom,Fem,Sing,Perf,NaN,Past,Part,Pass,NaN,NaN,NaN,NaN,NaN,acl,870
873,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,case,874
874,PRON,NaN,Loc,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,obl,872
875,PUNCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,punct,872
876,VERB,NaN,NaN,NaN,NaN,Perf,NaN,Past,Conv,Mid,NaN,NaN,NaN,NaN,NaN,acl,870
877,ADP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,case,878
878,NOUN,Inan,Acc,Fem,Sing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,obl,876


In [8]:
#--- prepare bigrams frame

import os
import sys
import pandas as pd
import pymorphy2

file_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'tg', 'grammar_ru', 'ml', 'features', 'bigrams.csv'))
bigrams = pd.read_csv(file_path, sep=" ")
bigrams = bigrams.drop('nans?', axis=1)
analyzer = pymorphy2.MorphAnalyzer()
bigrams['first_norm'] = bigrams.apply(lambda row: analyzer.parse(row['first'])[0].normal_form, axis=1)
bigrams['second_norm'] = bigrams.apply(lambda row: analyzer.parse(row.second)[0].normal_form, axis=1)

In [9]:
#--- count bigrams for replaced pronoun
import numpy as np

def filter(df, filter_df, required_col_name, index_col_name):
    col = df[df.index.isin(filter_df[index_col_name])][required_col_name]
    return (col.to_frame().reset_index().rename(columns={'word_id': index_col_name}))

def get_pair(first, second):
    return (bigrams['first_norm'].isin(pairs[first])) & (bigrams['second_norm'].isin(pairs[second]))

pairs = merged_df[['pronoun_word_id', 'candidate_word_id', 'pronoun_parent_id']]     
candidates = filter(morphology_df, pairs, 'normal_form', 'candidate_word_id')
parents = filter(morphology_df, pairs, 'normal_form', 'pronoun_parent_id')
pairs = pairs.merge(candidates, on='candidate_word_id')
pairs = pairs.merge(parents, on='pronoun_parent_id')
dict_pairs = bigrams[get_pair('normal_form_x', 'normal_form_y') | get_pair('normal_form_y', 'normal_form_x')]
pairs = pairs.merge(dict_pairs, how='left', left_on=['normal_form_x', 'normal_form_y'], right_on=['first_norm', 'second_norm']) 
pairs = pairs.merge(dict_pairs, how='left', left_on=['normal_form_x', 'normal_form_y'], right_on=['second_norm', 'first_norm'])
pairs['dict_bigrams_count'] = pairs['abs_y'].replace(np.nan, 0) + pairs['abs_x'].replace(np.nan, 0)
result = pairs[['pronoun_word_id', 'candidate_word_id', 'dict_bigrams_count']]
merged_df = merged_df.merge(result, on=['pronoun_word_id', 'candidate_word_id'])
merged_df

,pronoun_case,pronoun_word_id,candidate_POS,candidate_animacy,candidate_case,candidate_word_id,candidate_distance,pronoun_parent_id,dict_bigrams_count
0,accs,10,NOUN,anim,nomn,0,0,9,0.0
1,nomn,18,NOUN,anim,nomn,0,1,19,0.0
2,nomn,18,NPRO,NaN,accs,10,0,19,0.0
3,datv,30,NOUN,anim,nomn,0,2,31,0.0
4,datv,30,NPRO,NaN,accs,10,1,31,5.0
...,...,...,...,...,...,...,...,...,...
861,nomn,921,NOUN,anim,nomn,907,4,922,0.0
862,nomn,921,NOUN,inan,nomn,914,3,922,0.0
863,nomn,921,NPRO,NaN,datv,917,2,922,0.0
864,nomn,921,NOUN,inan,loct,918,1,922,0.0


In [10]:
merged_df.head(30)

,pronoun_case,pronoun_word_id,candidate_POS,candidate_animacy,candidate_case,candidate_word_id,candidate_distance,pronoun_parent_id,dict_bigrams_count
0,accs,10,NOUN,anim,nomn,0,0,9,0.0
1,nomn,18,NOUN,anim,nomn,0,1,19,0.0
2,nomn,18,NPRO,NaN,accs,10,0,19,0.0
3,datv,30,NOUN,anim,nomn,0,2,31,0.0
4,datv,30,NPRO,NaN,accs,10,1,31,5.0
5,datv,30,NPRO,NaN,accs,10,1,31,4.0
6,datv,30,NPRO,NaN,nomn,18,0,31,5.0
7,datv,30,NPRO,NaN,nomn,18,0,31,4.0
8,gent,48,ADJF,NaN,gent,3,7,46,0.0
9,gent,48,ADJF,NaN,gent,5,6,46,0.0
